In [1]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


In [40]:

driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://www.amazon.com.br'
driver.get(url)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/brunoozaki/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [41]:
def get_url(search_term):
    template = "https://www.amazon.com.br/s?k={}"
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

In [42]:
url = get_url('data science')
print(url)
driver.get(url)

https://www.amazon.com.br/s?k=data+science


In [43]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [60]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [61]:
len(results)

56

In [62]:
item = results[4]

In [63]:
atag = item.h2.a

In [64]:
description  = atag.text.strip()

In [65]:
url = 'https://www.amazon.com.br' + atag.get('href')

In [69]:
price_parent = item.find('span', 'a-price')

In [70]:
price = price_parent.find('span', 'a-offscreen').text

In [71]:
rating = item.i.text

In [74]:
rating

'4,7 de 5 estrelas'

In [80]:
review_count = item.find('span', 'a-size-base s-underline-text').text

In [81]:
review_count

'657'

In [84]:
def extract_record(item):
    atag = item.h2.a
    description  = atag.text.strip()
    url = 'https://www.amazon.com.br' + atag.get('href')


    try: 
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text

    except AttributeError:
        return 

    try:
        rating = item.i.text
        review_count = item.find('span', 'a-size-base s-underline-text').text
    except AttributeError:
        rating = ''
        review_count = ''

    result = (description, price, rating, review_count, url)

    return result

In [85]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [86]:
records

[('Homo Deus',
  'R$\xa044,90',
  '4,8 de 5 estrelas',
  '7.023',
  'https://www.amazon.com.br/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A05156301SPFV8GV46A0O&url=%2FHomo-deus-Yuval-Noah-Harari%2Fdp%2F8535928197%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Ddata%2Bscience%26qid%3D1649202374%26sr%3D8-1-spons%26psc%3D1&qualifier=1649202374&id=2261315069424878&widgetName=sp_atf'),
 ('21 lições para o século 21',
  'R$\xa046,90',
  '4,8 de 5 estrelas',
  '6.217',
  'https://www.amazon.com.br/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A05151063MUK2I6ZSDSF6&url=%2F21-li%25C3%25A7%25C3%25B5es-para-s%25C3%25A9culo%2Fdp%2F8535930914%2Fref%3Dsr_1_2_sspa%3Fkeywords%3Ddata%2Bscience%26qid%3D1649202374%26sr%3D8-2-spons%26psc%3D1&qualifier=1649202374&id=2261315069424878&widgetName=sp_atf'),
 ('Breve história de quase tudo',
  'R$\xa056,90',
  '4,7 de 5 estrelas',
  '484',
  'https://www.amazon.com.br/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf

In [ ]:
def get_url(search_term):
    template = "https://www.amazon.com.br/s?k={}"
    search_term = search_term.replace(' ', '+')

    url = template.format(search_term)

    url += '&page{}'
    return url

In [89]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

def get_url(search_term):
    template = "https://www.amazon.com.br/s?k={}"
    search_term = search_term.replace(' ', '+')

    url = template.format(search_term)

    url += '&page{}'
    return url

def extract_record(item):
    atag = item.h2.a
    description  = atag.text.strip()
    url = 'https://www.amazon.com.br' + atag.get('href')


    try: 
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text

    except AttributeError:
        return 

    try:
        rating = item.i.text
        review_count = item.find('span', 'a-size-base s-underline-text').text
    except AttributeError:
        rating = ''
        review_count = ''

    result = (description, price, rating, review_count, url)

    return result

def main(search_term):
    driver = webdriver.Chrome(ChromeDriverManager().install())


    record = []
    url = get_url(search_term)

    for page in (1, 8):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser') 
        results = soup.find_all('div', {'data-component-type': 's-search-result'})

        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)

    driver.close()

    with open('results.csv', 'w', newline= '', encoding= 'utf-8' ) as f:
        writer = csv.writer(f, delimiter = ';')
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)






In [90]:
main('data science')



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/brunoozaki/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


TypeError: 'enconding' is an invalid keyword argument for open()